In [64]:
import pandas as pd

In [65]:
df = pd.read_csv('/Users/sarap/Desktop/Data_Science/Capstone/Sample_Size_300.csv')
print ("Open CSV")

Open CSV


In [66]:
df.head()

,HexagonNumber,HexandMonth,Month,Under20,20to29,30to39,40to49,50to59,60to69,70andabove,...,MiscCals,OverdoseCalls,PropertyCalls,QOLCalls,TrafficCalls,ViolentCalls,AverageTemperature,AverageHumidity,AveragePrecipitation,Unemployment
0,1,1-2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.2740
1,1,1-4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.2740
2,2,2-11,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.1415
3,2,2-7,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.2740
4,2,2-1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.2740


In [67]:
#Drop the columns for HexandMonth and TotalNarcan.  HEXandMonth is not needed and TotalNarcan may cause bias because
#the Narcan Administered already accounts for the TotalNarcan for the month. 

df= df.drop(['HexandMonth'], axis=1)
df = df.drop(['TotalNarcan'], axis=1)
df = df.drop(['HexagonNumber'], axis = 1)
df = df.drop(['Month'], axis = 1)

In [68]:
df.columns

Index(['Under20', '20to29', '30to39', '40to49', '50to59', '60to69',
       '70andabove', 'NarcanAdministered', 'Male', 'Female', 'MPDCalls',
       'FEMSCalls', 'MedicalCalls', 'MiscCals', 'OverdoseCalls',
       'PropertyCalls', 'QOLCalls', 'TrafficCalls', 'ViolentCalls',
       'AverageTemperature', 'AverageHumidity', 'AveragePrecipitation',
       'Unemployment'],
      dtype='object')

In [123]:
from sklearn.model_selection import train_test_split as tts 

features = df [[
     '20to29', '30to39', '40to49',
       '50to59', '60to69', 
]]

X = features

y = df['NarcanAdministered']

print(X.shape)
print(y.shape)

(600, 5)
(600,)


In [124]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [125]:
from sklearn.metrics import classification_report

In [126]:
# Import the model form
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier() 
model.fit(X_test, y_test)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98        59
          1       1.00      0.97      0.98        61

avg / total       0.98      0.98      0.98       120



In [127]:
model = DecisionTreeClassifier(max_depth=5) 
model.fit(X_test, y_test)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98        59
          1       1.00      0.97      0.98        61

avg / total       0.98      0.98      0.98       120



In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
# We'll select 50 trees and opt for 'out-of-bag' samples to estimate the generalization error.
rf = RandomForestClassifier(n_estimators=50, oob_score=True)

In [130]:
# Next split up the data with the 'train test split' method in the Cross Validation module
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

# ...and then run the 'fit' method to build a forest of trees
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [131]:
rf.score(X_test, y_test)

0.98333333333333328

In [224]:
from sklearn.model_selection import train_test_split as tts 

features = df [[
      '20to29', '30to39', '40to49', '50to59', '60to69',
       '70andabove','MedicalCalls', 'MiscCals', 'OverdoseCalls',
       'PropertyCalls', 'QOLCalls', 'TrafficCalls', 'ViolentCalls',
   
     ]]

X = features

y = df['NarcanAdministered']

print(X.shape)
print(y.shape)

(600, 13)
(600,)


In [225]:

from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline


def model_selection(X, y, estimator):
    """
    Test various estimators.
    """ 
    y = LabelEncoder().fit_transform(y.values.ravel())
    model = Pipeline([
         ('label_encoding', EncodeCategorical(X.keys())), 
         ('one_hot_encoder', OneHotEncoder()), 
         ('estimator', estimator)
    ])

    # Instantiate the classification model and visualizer
    model.fit(X, y)  
    
    expected  = y
    predicted = model.predict(X)
    
    # Compute and return the F1 score (the harmonic mean of precision and recall)
    return (f1_score(expected, predicted))

In [226]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None. 
    """
    
    def __init__(self, columns=None):
        self.columns  = [col for col in columns] 
        self.encoders = None
    
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode. 
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns 
        
        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns 
        }
        return self

    def transform(self, data):
        """
        Uses the encoders to transform a data frame. 
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])
        
        return output

In [227]:
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier

In [228]:
model_selection(X, y, LogisticRegressionCV())

0.99328859060402686

In [229]:
model_selection(X, y, LogisticRegression())

0.98819561551433399

In [230]:
from sklearn.naive_bayes import GaussianNB

In [231]:
clf = GaussianNB()
clf.fit(X, y)

GaussianNB(priors=None)

In [232]:
clf.predict_proba(X)

array([[  1.00000000e+00,   2.93830427e-17],
       [  1.00000000e+00,   2.93830427e-17],
       [  1.00000000e+00,   2.93830427e-17],
       ..., 
       [  0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00]])

In [233]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf. fit(X,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [234]:
clf.predict_proba(X)

array([[  5.00000000e-01,   5.00000000e-01],
       [  5.00000000e-01,   5.00000000e-01],
       [  5.00000000e-01,   5.00000000e-01],
       ..., 
       [  1.35416612e-02,   9.86458339e-01],
       [  5.20551063e-02,   9.47944894e-01],
       [  7.77336413e-06,   9.99992227e-01]])